In [1]:
pip install -U "ray[data,train,tune,serve]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


# Ray Tune을 이용한 하이퍼파라미터 튜닝

[Ray Tune](https://docs.ray.io/en/latest/tune.html) 은 분산 하이퍼파라미터 튜닝을 위한 업계 표준 도구입니다.
Ray Tune은 최신 하이퍼파라미터 검색 알고리즘을 포함하고 TensorBoard 및 기타 분석 라이브러리와 통합되며 기본적으로
[Ray 의 분산 기계 학습 엔진](https://ray.io/) 을 통해 학습을 지원합니다.

이 튜토리얼은 Ray Tune을 파이토치 학습 workflow에 통합하는 방법을 알려줍니다.
CIFAR10 이미지 분류기를 훈련하기 위해 [파이토치 문서에서 이 튜토리얼을](https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html) 확장할 것입니다.

아래와 같이 약간의 수정만 추가하면 됩니다.

1. 함수에서 데이터 로딩 및 학습 부분을 감싸두고,
2. 일부 네트워크 파라미터를 구성 가능하게 하고,
3. 체크포인트를 추가하고 (선택 사항),
4. 모델 튜닝을 위한 검색 공간을 정의합니다.


이 튜토리얼을 실행하기 위해 아래의 패키지가 설치되어 있는지 확인하세요:

-  ``ray[tune]``: 배포된 하이퍼파라미터 튜닝 라이브러리
-  ``torchvision``: 데이터 변형을 위해 필요


In [2]:
from functools import partial
import numpy as np
import os
import tempfile

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

from ray import tune, train
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

## Data loaders
data loader를 자체 함수로 감싸두고 전역 데이터 디렉토리로 전달합니다.
이런 식으로 서로 다른 실험들 간에 데이터 디렉토리를 공유할 수 있습니다.



In [3]:
def load_data(data_dir="./data"):
    # 데이터 전처리를 위한 변환(transform)을 정의
    # ToTensor를 사용하여 이미지 데이터를 PyTorch 텐서로 변환하고,
    # Normalize를 사용하여 이미지의 각 채널을 정규화.
    # 정규화는 평균이 0.5, 표준편차가 0.5가 되도록 설정.
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # CIFAR-10 훈련 데이터셋을 다운로드하고, 위에서 정의한 변환을 적용.
    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

## 구성 가능한 신경망
구성 가능한 파라미터만 튜닝이 가능합니다.
이 예시를 통해 fully connected layer 크기를 지정할 수 있습니다:



In [4]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 학습 함수
학습 스크립트를 ``train_cifar(config, checkpoint_dir=None, data_dir=None)`` 함수로 감싸둡니다.
``config`` 매개변수는 훈련할 하이퍼파라미터를 받습니다. ``checkpoint_dir`` 매개변수는 체크포인트를 복원하는 데 사용됩니다. ``data_dir`` 은 데이터를 읽고 저장하는 디렉토리를 지정하므로,
여러 실행들이 동일한 데이터 소스를 공유할 수 있습니다.

```python
net = Net(config["l1"], config["l2"])

if checkpoint_dir:
    model_state, optimizer_state = torch.load(
        os.path.join(checkpoint_dir, "checkpoint"))
    net.load_state_dict(model_state)
    optimizer.load_state_dict(optimizer_state)
```
또한, 옵티마이저의 학습률(learning rate)을 구성할 수 있습니다.

```python
optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
```
학습 데이터를 학습 및 검증 세트로 나눕니다. 따라서 데이터의 80%는 모델 학습에 사용하고, 나머지 20%에 대해 유효성 검사 및 손실을 계산합니다. 학습 및 테스트 세트를 반복하는 배치 크기도 구성할 수 있습니다.

### DataParallel을 이용한 GPU(다중)지원 추가
이미지 분류는 GPU를 사용할 때 이점이 많습니다. 운좋게도 Ray Tune에서 파이토치의 추상화를 계속 사용할 수 있습니다.
따라서 여러 GPU에서 데이터 병렬 훈련을 지원하기 위해 모델을 ``nn.DataParallel`` 으로 감쌀 수 있습니다.

```python
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
net.to(device)
```
``device`` 변수를 사용하여 사용 가능한 GPU가 없을 때도 학습이 가능한지 확인합니다.

```python
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
```
이 코드는 CPU들, 단일 GPU 및 다중 GPU에 대한 학습을 지원합니다.
특히 Ray는 [fractional-GPU](https://docs.ray.io/en/master/using-ray-with-gpus.html#fractional-gpus) 도 지원하므로
모델이 GPU 메모리에 적합한 상황에서는 테스트 간에 GPU를 공유할 수 있습니다.

### Ray Tune과 소통하기

가장 흥미로운 부분은 Ray Tune과의 소통입니다.

```python
with tune.checkpoint_dir(epoch) as checkpoint_dir:
    path = os.path.join(checkpoint_dir, "checkpoint")
    torch.save((net.state_dict(), optimizer.state_dict()), path)

tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
```
여기서 먼저 체크포인트를 저장한 다음 일부 메트릭을 Ray Tune에 다시 보냅니다. 특히, validation loss와 accuracy를
Ray Tune으로 다시 보냅니다. 그 후 Ray Tune은 이러한 메트릭을 사용하여 최상의 결과를 유도하는 하이퍼파라미터 구성을
결정할 수 있습니다. 이러한 메트릭들은 또한 리소스 낭비를 방지하기 위해 성능이 좋지 않은 실험을 조기에 중지하는 데 사용할 수 있습니다.

체크포인트 저장은 선택사항이지만 [Population Based Training](https://docs.ray.io/en/master/tune/tutorials/tune-advanced-tutorial.html)
과 같은 고급 스케줄러를 사용하려면 필요합니다. 또한 체크포인트를 저장하면 나중에 학습된 모델을 로드하고 평가 세트(test set)에서 검증할 수 있습니다.

### Full training function

전체 예제 코드는 다음과 같습니다.



In [5]:
def train_cifar(config, data_dir=None):
    # Net 모델 인스턴스를 생성합니다. l1과 l2는 구성에서 지정된 레이어 크기입니다.
    net = Net(config["l1"], config["l2"])

    # 디바이스 설정: CUDA가 사용 가능한 경우 GPU를 사용하고, 그렇지 않으면 CPU를 사용합니다.
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        # 사용 가능한 CUDA 장치가 여러 개인 경우, DataParallel을 사용하여 모델을 병렬로 실행합니다.
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # 손실 함수와 옵티마이저를 정의합니다.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    # 체크포인트가 있는 경우, 체크포인트에서 모델 상태와 옵티마이저 상태를 로드합니다.
    checkpoint = train.get_checkpoint()
    if checkpoint is not None:
        with checkpoint.as_directory() as checkpoint_dir:
            checkpoint_path = os.path.join(checkpoint_dir, "checkpoint")
            model_state, optimizer_state = torch.load(checkpoint_path)
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    # 학습 데이터셋을 로드하고, 학습 세트와 검증 세트로 분할합니다.
    trainset, _ = load_data(data_dir)
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])

    # DataLoader를 사용하여 학습 및 검증 데이터를 배치로 로드합니다.
    trainloader = DataLoader(train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2)
    valloader = DataLoader(val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=2)

    # 학습 루프
    for epoch in range(10):
        net.train()
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # 검증 루프
        net.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        # 검증 단계 후의 메트릭을 계산하고 ray.train.report를 사용하여 보고합니다.
        metrics = {"loss": val_loss / total, "accuracy": correct / total}
        train.report(metrics)

    print("Finished Training")

## Test set 정확도(accuracy)
일반적으로 머신러닝 모델의 성능은 모델 학습에 사용되지 않은 데이터를 사용해 테스트합니다.
Test set 또한 함수로 감싸둘 수 있습니다.



In [6]:
def test_accuracy(net, device="cpu"):
    # 학습 데이터셋과 테스트 데이터셋을 로드합니다. 여기서는 테스트 세트만 사용됩니다.
    trainset, testset = load_data()

    # 테스트 데이터셋을 DataLoader를 통해 로드합니다.
    # 배치 크기는 4로 설정하고, 데이터를 섞지 않으며, 병렬 처리를 위해 2개의 워커를 사용합니다.
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    # 올바르게 예측된 샘플 수와 전체 샘플 수를 추적하는 변수를 초기화합니다.
    correct = 0
    total = 0
    # 기울기를 계산할 필요가 없으므로, no_grad() 컨텍스트를 사용하여 메모리 사용량을 줄이고 연산 속도를 높입니다.
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # 데이터와 레이블을 설정된 디바이스(CPU 또는 CUDA)로 이동합니다.
            images, labels = images.to(device), labels.to(device)
            # 현재 배치의 이미지에 대해 모델의 출력(예측)을 계산합니다.
            outputs = net(images)
            # 출력에서 가장 높은 값(가장 확신하는 클래스)을 가진 인덱스를 찾습니다.
            _, predicted = torch.max(outputs.data, 1)
            # 전체 샘플 수에 이 배치의 샘플 수를 더합니다.
            total += labels.size(0)
            # 올바르게 예측된 샘플의 수를 더합니다. 예측과 실제 레이블이 일치하는 경우의 수입니다.
            correct += (predicted == labels).sum().item()

    # 전체 샘플 대비 올바르게 예측된 샘플의 비율(정확도)를 계산하여 반환합니다.
    return correct / total

이 함수는 또한 ``device`` 파라미터를 요구하므로, test set 평가를 GPU에서 수행할 수 있습니다.

## 검색 공간 구성
마지막으로 Ray Tune의 검색 공간을 정의해야 합니다. 예시는 아래와 같습니다.

```python
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}
```
``tune.sample_from()`` 함수를 사용하면 고유한 샘플 방법을 정의하여 하이퍼파라미터를 얻을 수 있습니다.
이 예제에서 ``l1`` 과 ``l2`` 파라미터는 4와 256 사이의 2의 거듭제곱이어야 하므로 4, 8, 16, 32, 64, 128, 256입니다.
``lr`` (학습률)은 0.0001과 0.1 사이에서 균일하게 샘플링 되어야 합니다. 마지막으로, 배치 크기는 2, 4, 8, 16중에서 선택할 수 있습니다.

각 실험에서, Ray Tune은 이제 이러한 검색 공간에서 매개변수 조합을 무작위로 샘플링합니다.
그런 다음 여러 모델을 병렬로 훈련하고 이 중에서 가장 성능이 좋은 모델을 찾습니다. 또한 성능이 좋지 않은 실험을 조기에 종료하는 ``ASHAScheduler`` 를 사용합니다.

상수 ``data_dir`` 파라미터를 설정하기 위해 ``functools.partial`` 로 ``train_cifar`` 함수를 감싸둡니다. 또한 각 실험에 사용할 수 있는 자원들(resources)을 Ray Tune에 알릴 수 있습니다.

```python
gpus_per_trial = 2
# ...
result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    checkpoint_at_end=True)
```
파이토치 ``DataLoader`` 인스턴스의 ``num_workers`` 을 늘리기 위해 CPU 수를 지정하고 사용할 수 있습니다.
각 실험에서 선택한 수의 GPU들은 파이토치에 표시됩니다. 실험들은 요청되지 않은 GPU에 액세스할 수 없으므로 같은 자원들을 사용하는 중복된 실험에 대해 신경쓰지 않아도 됩니다.

부분 GPUs를 지정할 수도 있으므로, ``gpus_per_trial=0.5`` 와 같은 것 또한 가능합니다. 이후 각 실험은 GPU를 공유합니다. 사용자는 모델이 여전히 GPU메모리에 적합한지만 확인하면 됩니다.

모델을 훈련시킨 후, 가장 성능이 좋은 모델을 찾고 체크포인트 파일에서 학습된 모델을 로드합니다. 이후 test set 정확도(accuracy)를 얻고 모든 것들을 출력하여 확인할 수 있습니다.

전체 주요 기능은 다음과 같습니다.



In [7]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    # 데이터셋이 위치할 디렉토리의 절대 경로를 설정합니다.
    data_dir = os.path.abspath("./data")
    # 데이터셋을 로드합니다.
    load_data(data_dir)

    # 하이퍼파라미터 설정을 위한 구성입니다.
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),  # 첫 번째 레이어의 유닛 수
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),  # 두 번째 레이어의 유닛 수
        "lr": tune.loguniform(1e-4, 1e-1),  # 학습률
        "batch_size": tune.choice([2, 4, 8, 16])  # 배치 크기
    }

    # ASHAScheduler를 사용하여 조기 중단을 구성합니다.
    scheduler = ASHAScheduler(
        metric="loss",  # 최적화할 메트릭
        mode="min",  # 메트릭을 최소화
        max_t=max_num_epochs,  # 최대 에폭 수
        grace_period=1,  # 조기 중단 전 최소 에폭 수
        reduction_factor=2)  # 트라이얼 감소 비율

    # CLIReporter를 사용하여 트라이얼 진행 상황을 터미널에 보고합니다.
    reporter = CLIReporter(metric_columns=["loss", "accuracy", "training_iteration"])

    # tune.run을 사용하여 학습을 실행합니다.
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),  # 학습 함수
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},  # 트라이얼당 리소스
        config=config,
        num_samples=num_samples,  # 샘플링할 구성의 수
        scheduler=scheduler,
        progress_reporter=reporter)

    # 가장 좋은 트라이얼을 선택합니다.
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))

    # 가장 좋은 트라이얼의 모델을 로드합니다.
    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    # 체크포인트에서 모델 상태를 로드합니다.
    if best_trial.checkpoint:
        with best_trial.checkpoint.as_directory() as best_checkpoint_dir:
            model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
            best_trained_model.load_state_dict(model_state)

    # 테스트 데이터셋에서 모델의 정확도를 평가합니다.
    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=1)

100%|██████████| 170498071/170498071 [00:04<00:00, 34626749.68it/s]


Extracting /content/data/cifar-10-python.tar.gz to /content/data
Files already downloaded and verified


2024-05-22 00:44:42,393	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2024-05-22 00:44:46,057	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-05-22 00:44:46,893	WARNING tune.py:900 -- AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


+--------------------------------------------------------------------+
| Configuration for experiment     train_cifar_2024-05-22_00-44-46   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        AsyncHyperBandScheduler           |
| Number of trials                 10                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_cifar_2024-05-22_00-44-46
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-05-22_00-44-36_381171_1581/artifacts/2024-05-22_00-44-46/train_cifar_2024-05-22_00-44-46/driver_artifacts`

Trial status: 10 PENDING
Current time: 2024-05-22 00:44:47. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------+

2024-05-22 00:56:25,046	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-05-22 00:56:25,056	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-05-22 00:56:25,063	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_cifar_2024-05-22_00-44-46' in 0.0104s.

Trial status: 1 RUNNING | 9 PENDING
Current time: 2024-05-22 00:56:25. Total running time: 11min 38s
Logical resource usage: 2.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------------------------------------+
| Trial name                status              lr     batch_size     iter     total time (s)      loss     accuracy |
+--------------------------------------------------------------------------------------------------------------------+
| train_cifar_7c8c5_00000   RUNNING    0.0331583                2        7            653.081   1.15761       0.0996 |
| train_cifar_7c8c5_00001   PENDING    0.00322777               4                                                    |
| train_cifar_7c8c5_00002   PENDING    0.000446491              2                                                    |
| train_cifar_7c8c5_00003   PENDING    0.000242495              8                                           

2024-05-22 00:56:35,099	WARNING tune.py:1054 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-05-22 00:56:35,117	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 9 trial(s):
- train_cifar_7c8c5_00001: FileNotFoundError('Could not fetch metrics for train_cifar_7c8c5_00001: both result.json and progress.csv were not found at /root/ray_results/train_cifar_2024-05-22_00-44-46/train_cifar_7c8c5_00001_1_batch_size=4,lr=0.0032_2024-05-22_00-44-47')
- train_cifar_7c8c5_00002: FileNotFoundError('Could not fetch metrics for train_cifar_7c8c5_00002: both result.json and progress.csv were not found at /root/ray_results/train_cifar_2024-05-22_00-44-46/train_cifar_7c8c5_00002_2_batch_size=2,lr=0.0004_2024-05-22_00-44-47')
- train_cifar_7c8c5_00003: FileNotFoundError('Could not fetch metrics for train_cifar_7c8c5_00003: both result.json and progress.csv were not found at /root/ray_results/train_cifar_2024


Best trial config: {'l1': 64, 'l2': 128, 'lr': 0.03315830989668164, 'batch_size': 2}
Best trial final validation loss: 1.1576066904306412
Best trial final validation accuracy: 0.0996
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.0994


코드를 실행하면 결과는 다음과 같습니다.

::

    Number of trials: 10 (10 TERMINATED)
    +-----+------+------+-------------+--------------+---------+------------+--------------------+
    | ... |   l1 |   l2 |          lr |   batch_size |    loss |   accuracy | training_iteration |
    |-----+------+------+-------------+--------------+---------+------------+--------------------|
    | ... |   64 |    4 | 0.00011629  |            2 | 1.87273 |     0.244  |                  2 |
    | ... |   32 |   64 | 0.000339763 |            8 | 1.23603 |     0.567  |                  8 |
    | ... |    8 |   16 | 0.00276249  |           16 | 1.1815  |     0.5836 |                 10 |
    | ... |    4 |   64 | 0.000648721 |            4 | 1.31131 |     0.5224 |                  8 |
    | ... |   32 |   16 | 0.000340753 |            8 | 1.26454 |     0.5444 |                  8 |
    | ... |    8 |    4 | 0.000699775 |            8 | 1.99594 |     0.1983 |                  2 |
    | ... |  256 |    8 | 0.0839654   |           16 | 2.3119  |     0.0993 |                  1 |
    | ... |   16 |  128 | 0.0758154   |           16 | 2.33575 |     0.1327 |                  1 |
    | ... |   16 |    8 | 0.0763312   |           16 | 2.31129 |     0.1042 |                  4 |
    | ... |  128 |   16 | 0.000124903 |            4 | 2.26917 |     0.1945 |                  1 |
    +-----+------+------+-------------+--------------+---------+------------+--------------------+


    Best trial config: {'l1': 8, 'l2': 16, 'lr': 0.00276249, 'batch_size': 16, 'data_dir': '...'}
    Best trial final validation loss: 1.181501
    Best trial final validation accuracy: 0.5836
    Best trial test set accuracy: 0.5806

대부분의 실험은 자원 낭비를 막기 위해 일찍 중단되었습니다. 가장 좋은 결과를 얻은 실험은 58%의 정확도를 달성했으며, 이는 테스트 세트에서 확인할 수 있습니다.

이제 파이토치 모델의 매개변수를 조정할 수 있습니다.


